In [1]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_community.chat_models import ChatOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain_openai import AzureOpenAI 
import pandas as pd
import dice_ml
from pathlib import Path
from zeroshot_prompt import *

import os

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import tensorflow as tf
from keras import optimizers, Sequential
from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Dropout, Input
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
from keras.models import Model, load_model
from keras import regularizers
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, auc, roc_curve
import os
from tqdm import tqdm

LABELS = ["Normal", "Anomaly"]

### 🟢 Step 1: Data Preprocessing Functions ###
def read_data(path):
    """Reads and preprocesses dataset."""
    df1 = pd.read_csv(path)
    df1 = df1.drop(['start_ts', 'session_duration'], axis=1)
    df1 = df1.fillna(0)
    df1['role'] = df1['role'].astype('category').cat.codes
    df1['user'] = df1['user'].astype('category').cat.codes
    return df1

def get_train_test_data(train, test, lookback):
    """Prepares training and testing sequences."""
    sc = MinMaxScaler(feature_range=(0, 1))
    train_scaled = sc.fit_transform(train.drop(columns=['class', 'type'], errors='ignore'))
    test_scaled = sc.transform(test.drop(columns=['class', 'type'], errors='ignore'))

    # Creating time-series sequences
    X_train = np.array([train_scaled[i - lookback:i, :] for i in range(lookback, len(train_scaled))])
    X_test = np.array([test_scaled[i - lookback:i, :] for i in range(lookback, len(test_scaled))])
    y_test = test.iloc[lookback - 1:, test.columns.get_loc('class')].values

    return X_train, X_test, y_test, sc

### 🟢 Step 2: Counterfactual Generation Function ###

def generate_diverse_counterfactuals(
    model, sequence, scaler, feature_names, num_counterfactuals=5, 
    learning_rate=0.01, iterations=500, threshold=0.04, immutable_features=[], diversity_weight=0.01
):
    """
    Generates diverse counterfactuals while ensuring they are classified as normal 
    (i.e., reconstruction error is below the given threshold). A diversity term is added 
    to the loss to encourage counterfactuals to be different from one another.
    """
    counterfactuals = []
    print("\n🚀 Generating Counterfactuals...\n")

    # Convert the input sequence to a TensorFlow tensor.
    sequence_tf = tf.convert_to_tensor(sequence, dtype=tf.float32)
    
    # Determine indices for immutable features.
    immutable_indices = np.array(
        [feature_names.index(feature) for feature in immutable_features if feature in feature_names], dtype=int
    )
    original_immutable_values = sequence[:, :, immutable_indices].astype(np.float32)

    for cf_idx in tqdm(range(num_counterfactuals), desc="Generating Counterfactuals", unit="cf"):
        # Initialize the candidate counterfactual with the original sequence.
        seq_cf = tf.Variable(sequence_tf, dtype=tf.float32)
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

        for _ in tqdm(range(iterations), desc=f"Optimizing CF {cf_idx+1}/{num_counterfactuals}", unit="step", leave=False):
            with tf.GradientTape() as tape:
                tape.watch(seq_cf)
                reconstruction = model(seq_cf)
                reconstruction_loss = tf.reduce_mean(tf.abs(reconstruction - seq_cf))
                
                # Apply a penalty if reconstruction error is above the threshold.
                penalty = tf.maximum(reconstruction_loss - threshold, 0) * 10.0

                # Compute diversity loss: for each already finalized counterfactual,
                # penalize if the current candidate is too similar.
                diversity_loss = 0.0
                if len(counterfactuals) > 0:
                    for prev in counterfactuals:
                        prev_tensor = tf.convert_to_tensor(prev, dtype=tf.float32)
                        distance = tf.norm(seq_cf - prev_tensor)
                        diversity_loss += 1.0 / (distance + 1e-8)  # Avoid division by zero

                # Total loss includes reconstruction loss, anomaly penalty, and diversity term.
                total_loss = reconstruction_loss + penalty + diversity_weight * diversity_loss

            grads = tape.gradient(total_loss, seq_cf)
            if grads is not None:
                grads_numpy = grads.numpy()
                # Zero out gradients for immutable features so they remain unchanged.
                if len(immutable_indices) > 0:
                    grads_numpy[:, :, immutable_indices] = 0  
                grads_tf = tf.convert_to_tensor(grads_numpy, dtype=tf.float32)
                optimizer.apply_gradients([(grads_tf, seq_cf)])

                # Clip the candidate values to [0, 1].
                seq_cf.assign(tf.clip_by_value(seq_cf, 0, 1))

                # Restore immutable features using their original values.
                updates = tf.convert_to_tensor(original_immutable_values, dtype=tf.float32)
                indices = np.array([[b, t, f] 
                                     for b in range(seq_cf.shape[0]) 
                                     for t in range(seq_cf.shape[1]) 
                                     for f in immutable_indices])
                seq_cf.assign(tf.tensor_scatter_nd_update(seq_cf, indices, tf.reshape(updates, [-1])))

            # Early stopping if the candidate's reconstruction error is below the threshold.
            if reconstruction_loss.numpy() < threshold:
                print(f"✅ CF {cf_idx+1} is now normal (MSE={reconstruction_loss.numpy():.5f}) - stopping early")
                break

        counterfactuals.append(seq_cf.numpy())

    print("\n✅ Counterfactual Generation Complete!")

    # Reshape and inverse-transform to get the counterfactuals back on the original scale.
    counterfactuals = np.array(counterfactuals)
    reshaped_cf = counterfactuals.reshape(-1, sequence.shape[-1])
    counterfactuals_original_scale = scaler.inverse_transform(reshaped_cf)
    counterfactuals_original_scale = counterfactuals_original_scale.reshape(num_counterfactuals, sequence.shape[1], sequence.shape[2])

    return [pd.DataFrame(cf, columns=feature_names) for cf in counterfactuals_original_scale]



### 🟢 Step 3: Model Evaluation Function ###
def evaluate_model_on_sequences(model, anomalous_sequence, counterfactual_sequences, feature_names, scaler):
    """
    Evaluates both the original anomaly and generated counterfactuals.
    """
    anomaly_reconstructed = model.predict(anomalous_sequence)
    anomaly_error = np.mean(np.abs(anomalous_sequence - anomaly_reconstructed))

    evaluation_results = []
    for cf_sequence in counterfactual_sequences:
        cf_sequence = scaler.transform(cf_sequence)
        cf_sequence = cf_sequence.reshape(1, anomalous_sequence.shape[1], anomalous_sequence.shape[2])
        cf_reconstructed = model.predict(cf_sequence)
        cf_error = np.mean(np.abs(cf_sequence - cf_reconstructed))

        anomaly_original = scaler.inverse_transform(anomalous_sequence.reshape(-1, len(feature_names)))
        counterfactual_original = scaler.inverse_transform(cf_sequence.reshape(-1, len(feature_names)))
        feature_differences = np.abs(anomaly_original - counterfactual_original)
        diff_df = pd.DataFrame(feature_differences, columns=feature_names)
        mean_differences = diff_df.mean().sort_values(ascending=False)
        sorted_feature_differences = pd.DataFrame(mean_differences, columns=['Difference'])

        evaluation_results.append({
            "counterfactual_reconstruction_error": cf_error,
            "feature_differences": sorted_feature_differences
        })

    print("\n🔍 **Model Evaluation Results:**")
    print(f"🛑 Anomalous Session Reconstruction Error: {anomaly_error:.6f}")

    for idx, result in enumerate(evaluation_results):
        print(f"\n✅ Counterfactual {idx+1} Reconstruction Error: {result['counterfactual_reconstruction_error']:.6f}")
        print(result["feature_differences"].to_string())
    return evaluation_results

### 🟢 Step 4: Main Execution ###

path = "/home/sathish/UEBA/data/data.csv"
df = read_data(path)
train_data, test_data = df.iloc[:276388], df.iloc[276388:]
lookback = 3
X_train, X_test, y_test, scaler = get_train_test_data(train_data, test_data, lookback)
n_features = X_train.shape[2]

# Load or train model
model_path = "lstm_autoencoder.h5"
lstm_model = load_model(model_path, custom_objects={'MeanSquaredError': tf.keras.losses.MeanSquaredError()}) if os.path.exists(model_path) else None

mse = np.mean(np.power(X_test - lstm_model.predict(X_test), 2), axis=(1, 2))
threshold = 0.04
anomaly_idx = np.where(mse > threshold)[0][0]
anomalous_sequence = X_test[anomaly_idx].reshape(1, lookback, X_train.shape[2])

feature_names = train_data.drop(columns=['class', 'type'], errors='ignore').columns.tolist()
# Set threshold for counterfactual generation to 0.02
counterfactual_examples = generate_diverse_counterfactuals(
    lstm_model,
    anomalous_sequence,
    scaler,
    feature_names,
    threshold=0.03,
    immutable_features=["user", "role", "O", "C", "E", "A", "N"]
)
evaluate_model_on_sequences(lstm_model, anomalous_sequence, counterfactual_examples, feature_names, scaler)




In [34]:
import json
def format_data_for_llm(anomalous_sequence, counterfactual_examples, feature_names):
    """
    Formats the anomaly and counterfactuals into a structured format for LLM input.
    """

    anomaly_data = pd.DataFrame(anomalous_sequence.reshape(3, len(feature_names)), columns=feature_names)

    counterfactuals_data = [
        pd.DataFrame(cf.values.reshape(3, len(feature_names)), columns=feature_names).to_dict(orient="records")
        for cf in counterfactual_examples
    ]

    formatted_data = {
        "anomalous_session": anomaly_data.to_dict(orient="records"),
        "counterfactuals": counterfactuals_data,
    }

    return json.dumps(formatted_data, indent=4)


In [35]:
# ✅ **Format Data for LLM**
formatted_data = format_data_for_llm(anomalous_sequence, counterfactual_examples, feature_names)
    
print("\n🔹 **Formatted Data for LLM:**")
print(formatted_data)


🔹 **Formatted Data for LLM:**
{
    "anomalous_session": [
        {
            "user": 0.6756756756756757,
            "logon_on_own_pc_normal": 0.0,
            "logon_on_other_pc_normal": 0.0,
            "logon_on_own_pc_off_hour": 0.3333333333333333,
            "logon_on_other_pc_off_hour": 0.0,
            "logon_hour": 0.30434782608695654,
            "day_of_a_week": 0.16666666666666666,
            "device_connects_on_own_pc": 0.7000000000000001,
            "device_connects_on_other_pc": 0.0,
            "device_connects_on_own_pc_off_hour": 0.0,
            "device_connects_on_other_pc_off_hour": 0.0,
            "documents_copy_own_pc": 0.8148148148148148,
            "documents_copy_other_pc": 0.0,
            "exe_files_copy_own_pc": 0.0,
            "exe_files_copy_other_pc": 0.0,
            "documents_copy_own_pc_off_hour": 0.0,
            "documents_copy_other_pc_off_hour": 0.0,
            "exe_files_copy_own_pc_off_hour": 0.0,
            "exe_files_copy_other_p

In [3]:
#print(anomalous_sequence.shape)

co_original = scaler.inverse_transform(anomalous_sequence.reshape(-1, len(feature_names)))
#print(co_original, type(co_original))
#print(co_original[0])
co_original_df = pd.DataFrame(co_original.reshape(3,-1),columns=counterfactual_examples[0].columns)
print(co_original_df.to_string())

    user  logon_on_own_pc_normal  logon_on_other_pc_normal  logon_on_own_pc_off_hour  logon_on_other_pc_off_hour  logon_hour  day_of_a_week  device_connects_on_own_pc  device_connects_on_other_pc  device_connects_on_own_pc_off_hour  device_connects_on_other_pc_off_hour  documents_copy_own_pc  documents_copy_other_pc  exe_files_copy_own_pc  exe_files_copy_other_pc  documents_copy_own_pc_off_hour  documents_copy_other_pc_off_hour  exe_files_copy_own_pc_off_hour  exe_files_copy_other_pc_off_hour  neutral_sites  job_search  hacking_sites  neutral_sites_off_hour  job_search_off_hour  hacking_sites_off_hour  total_emails  int_to_int_mails  int_to_out_mails  out_to_int_mails  out_to_out_mails  internal_recipients  external_recipients  distinct_bcc  mails_with_attachments  after_hour_mails  role  business_unit  functional_unit  department  team     O     C     E     A     N
0  675.0                     0.0                       0.0                       1.0                         0.0         

In [4]:
#cf_example = pd.DataFrame(counterfactual_examples[0].iloc[0])
cf_example = counterfactual_examples
#cf_example1  = cf_example.T.round(1)
print(type(cf_example))
cf_example[4]   # list of CF dataframes


<class 'list'>


,user,logon_on_own_pc_normal,logon_on_other_pc_normal,logon_on_own_pc_off_hour,logon_on_other_pc_off_hour,logon_hour,day_of_a_week,device_connects_on_own_pc,device_connects_on_other_pc,device_connects_on_own_pc_off_hour,...,role,business_unit,functional_unit,department,team,O,C,E,A,N
0,675.0,0.713983,0.007962,0.535136,0.0,7.441973,1.977825,7.373900,0.015285,0.294130,...,14.0,1.0,2.837465,2.099065,3.843745,39.0,40.0,47.0,50.0,26.0
1,675.0,0.854455,0.014584,0.401808,0.0,7.579709,2.011969,7.277054,0.001431,0.266381,...,14.0,1.0,2.961633,2.349903,3.085686,39.0,40.0,47.0,50.0,26.0
2,675.0,0.837595,0.000000,0.403395,0.0,7.557063,1.978618,7.033497,0.010309,0.221397,...,14.0,1.0,2.922812,2.143360,2.789212,39.0,40.0,47.0,50.0,26.0


In [5]:
 # Initialize Azure OpenAI client
llm = AzureChatOpenAI(
            deployment_name="gpt-4o",
            openai_api_base=os.getenv("ENDPOINT_URL", "https://g4266-m3cff6ws-swedencentral.openai.azure.com/"),
            openai_api_key=os.getenv("AZURE_OPENAI_API_KEY", "6xxmkjbfwnm3Li1keZmIG0V5cggGUXPqfSdAppx2EkW0iUSXSbrDJQQJ99AKACfhMk5XJ3w3AAAAACOGdLLq"),
            openai_api_version="2024-05-01-preview"
        )


/tmp/ipykernel_575828/2128844656.py:2: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureChatOpenAI``.
  llm = AzureChatOpenAI(
/home/sathish/UEBA/.venv/lib/python3.10/site-packages/langchain_community/chat_models/azure_openai.py:174: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://g4266-m3cff6ws-swedencentral.openai.azure.com/ to https://g4266-m3cff6ws-swedencentral.openai.azure.com/openai.
  warnings.warn(
/home/sathish/UEBA/.venv/lib/python3.10/site-packages/langchain_community/chat_models/azure_openai.py:181: UserWarning: As of openai>=1.0.0, if `deployment_n

In [26]:
def ZeroShotExplain1():
    return  """
        Im providing a negative outcome from a {ML-system} and your task provide obeservations over the provided Insider Anomaly flagged by a LSTM model by comparing them to Counterfactuals generated from the Anomaly. 
        ----- Anomolous outcome -----
        {negative_outcome}

        
        ----- Positive couterfactual outcome -----
        {positive_outcome}


        ----- Observations -----
        <List of Observations>
        """


In [27]:

result = ZeroShotExplain1()
print(result)


        Im providing a negative outcome from a {ML-system} and your task provide obeservations over the provided Insider Anomaly flagged by a LSTM model by comparing them to Counterfactuals generated from the Anomaly. 
        ----- Anomolous outcome -----
        {negative_outcome}

        
        ----- Positive couterfactual outcome -----
        {positive_outcome}


        ----- Observations -----
        <List of Observations>
        


In [28]:
template0 = ZeroShotExplain1()
template1 = ZeroShotRules()
template2 = ZeroShotRulesCode()
template3 = ZeroShotExplanation(user_input=False)
template4 = ZeroShotExample()
template5 = ZeroShotExampleCode()

In [29]:
model_description = """ML-system that detects Anomolous behavior on a network"""

In [30]:
rules_chain = LLMChain(
            llm=llm,
            prompt=PromptTemplate(
                input_variables=["ML-system", "negative_outcome", "positive_outcome"],
                template=template0
            )
        )

In [31]:
response = rules_chain.run(({
        "ML-system": model_description,
        "negative_outcome": co_original_df.to_string,
        "positive_outcome": cf_example[4].to_string
        }))


In [32]:
print(response)

Here are the observations based on the anomalous outcome and the positive counterfactual outcome:

---

### **Logon Behavior**
1. **`logon_on_own_pc_normal`:**  
   - **Anomalous Outcome:** The user never logs on to their own PC during normal hours (`0.0` across all rows).  
   - **Counterfactual Outcome:** Indicates a significant increase in this behavior with values ranging between `0.713983` and `0.854455`.  
   - **Observation:** The lack of normal logon behavior on their own PC is flagged as anomalous. In the counterfactual, the system suggests that the user should occasionally log on to their own PC during normal hours to align with expected behavior patterns.

2. **`logon_on_other_pc_normal`:**  
   - **Anomalous Outcome:** The user never logs on to another PC during normal hours (`0.0` across all rows).  
   - **Counterfactual Outcome:** Suggests small but non-zero values (`0.007962` to `0.014584`).  
   - **Observation:** While the user does not log on to other PCs during norm